In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import glob
import pandas as pd

file_pattern = "/content/drive/MyDrive/ML/transformer/sms-call-internet-mi-2013-11-*.csv"


file_list = glob.glob(file_pattern)

data = []

for file_path in file_list:
    d = pd.read_csv(file_path)
    print(len(d))
    data.append(d)
dataset = pd.concat(data, ignore_index=True)
print(len(dataset))

5639488
5874140
5912097
5886883
5918856
5052416
4731582
39015462


In [ ]:
print(f"Labels (columns) in dataset: {dataset.columns.tolist()}")

Labels (columns) in dataset: ['CellID', 'datetime', 'countrycode', 'smsin', 'smsout', 'callin', 'callout', 'internet']


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the Milan Telecom dataset
def process_data(data):
    # Encode categorical variables if necessary
    label_encoders = {}
    for col in ['CellID', 'countrycode']:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

    # Convert datetime to numerical features
    data['datetime'] = pd.to_datetime(data['datetime'])
    data['hour'] = data['datetime'].dt.hour
    data['dayofweek'] = data['datetime'].dt.dayofweek
    data.drop(columns=['datetime'], inplace=True)

    return data, label_encoders

# Define Dataset class
class TelecomDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length

        # Normalize continuous variables
        self.scaler = StandardScaler()
        self.data.iloc[:, 2:] = self.scaler.fit_transform(self.data.iloc[:, 2:])

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        seq = self.data.iloc[idx:idx + self.sequence_length].values
        target = self.data.iloc[idx + self.sequence_length].values
        return torch.tensor(seq, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)

# Define iTransformer model
class iTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, ff_dim, num_layers, output_dim, dropout=0.1):
        super(iTransformer, self).__init__()

        self.embedding = nn.Linear(input_dim, model_dim)
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=model_dim,
                nhead=num_heads,
                dim_feedforward=ff_dim,
                dropout=dropout,
                activation="relu"
            ) for _ in range(num_layers)
        ])
        self.layer_norm = nn.LayerNorm(model_dim)
        self.projection = nn.Linear(model_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        for layer in self.encoder_layers:
            x = layer(x)
        x = self.layer_norm(x)
        x = self.projection(x[:, -1, :])  # Use the last time step for forecasting
        return x

# Hyperparameters
hyperparams = {
    "sequence_length": 24,
    "batch_size": 64,
    "epochs": 50,
    "learning_rate": 0.001,
    "model_dim": 128,
    "num_heads": 8,
    "ff_dim": 256,
    "num_layers": 4,
    "dropout": 0.1,
}

# Training function
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for seq, target in train_loader:
        seq, target = seq.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(seq)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Main function
def main():
    data, label_encoders = process_data(dataset)

    telecom_dataset = TelecomDataset(data, hyperparams['sequence_length'])
    train_loader = DataLoader(telecom_dataset, batch_size=hyperparams['batch_size'], shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = iTransformer(
        input_dim=data.shape[1],
        model_dim=hyperparams['model_dim'],
        num_heads=hyperparams['num_heads'],
        ff_dim=hyperparams['ff_dim'],
        num_layers=hyperparams['num_layers'],
        output_dim=data.shape[1],
        dropout=hyperparams['dropout']
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams['learning_rate'])

    for epoch in range(hyperparams['epochs']):
        train_loss = train_model(model, train_loader, criterion, optimizer, device)
        print(f"Epoch {epoch + 1}/{hyperparams['epochs']}, Loss: {train_loss:.4f}")

if __name__ == "__main__":
    main()
